# 1 - Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parents[0]))

In [3]:
import pandas as pd
import numpy as np

In [4]:
from src.utils import templates, config, io

/Users/hippolytegrandet/Desktop/Dev/country_risk_rating/.venv/lib/python3.9/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


# 2 - Intigration

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression

In [8]:
dataset = io.load_csv(config.PROCESSED_DATA_DIR / 'final_dataset.csv', index_col=0)
dataset

,YEAR,ISO3_COUNTRY_CODE,OECD_RATING,WB-WDI-NY.ADJ.DPEM.GN.ZS,IMF-FM-GGR_G01_GDP_PT,IMF-FM-G_X_G01_GDP_PT,WB-WDI-NY.GDP.PCAP.CD,Info-Country_Name,Legal_Systems-Civil_Law,Legal_Systems-Common_Law,...,Languages-Widely_spoken,Geography-x_coord,Geography-y_coord,Geography-Region,Geography-Sub_Region,Geography-Intermediate_Region,Geography-Region_Code,Geography-Sub_Region_Code,Geography-Intermediate_Region_Code,Economy-Income_Group
COUNTRY_PERIOD_INDEX,,,,,,,,,,,,,,,,,,,,,
AFG-1999,1999,AFG,7,NaN,NaN,NaN,NaN,AFGHANISTAN,0.0,0.0,...,NaN,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2000,2000,AFG,7,NaN,NaN,NaN,180.188369,AFGHANISTAN,0.0,0.0,...,NaN,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2001,2001,AFG,-,NaN,NaN,NaN,142.903364,AFGHANISTAN,0.0,0.0,...,NaN,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2002,2002,AFG,-,NaN,NaN,NaN,182.174038,AFGHANISTAN,0.0,0.0,...,NaN,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2003,2003,AFG,-,NaN,9.825829,11.927498,199.643227,AFGHANISTAN,0.0,0.0,...,NaN,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE-2022,2022,ZWE,7,NaN,16.598777,21.476258,1676.821489,ZIMBABWE,1.0,1.0,...,Shona/Ndebele,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income
ZWE-2023,2023,ZWE,7,NaN,14.648601,19.968292,1592.416574,ZIMBABWE,1.0,1.0,...,Shona/Ndebele,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income
ZWE-2024,2024,ZWE,-,NaN,16.581003,18.640867,NaN,ZIMBABWE,1.0,1.0,...,Shona/Ndebele,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income


In [13]:

def filter_missingness(
    X: pd.DataFrame,
    max_missing_ratio: float = 0.3
) -> pd.DataFrame:
    missing_ratio = X.isna().mean()
    keep_cols = missing_ratio[missing_ratio <= max_missing_ratio].index
    return X[keep_cols]


def filter_low_variance(
    X: pd.DataFrame,
    threshold: float = 1e-4
) -> pd.DataFrame:
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(X.fillna(0))
    return X.loc[:, selector.get_feature_names_out()]


def filter_correlated(
    X: pd.DataFrame,
    max_corr: float = 0.9
) -> pd.DataFrame:
    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    drop_cols = [c for c in upper.columns if any(upper[c] > max_corr)]
    return X.drop(columns=drop_cols)


def select_by_mutual_information(
    X: pd.DataFrame,
    y: pd.Series,
    top_k: int = 100
) -> pd.DataFrame:
    mi = mutual_info_regression(X.fillna(0), y)
    scores = pd.Series(mi, index=X.columns)
    keep = scores.sort_values(ascending=False).head(top_k).index
    return X[keep]


In [15]:
filter_missingness(dataset, max_missing_ratio=0.25)

,YEAR,ISO3_COUNTRY_CODE,OECD_RATING,IMF-FM-GGR_G01_GDP_PT,IMF-FM-G_X_G01_GDP_PT,WB-WDI-NY.GDP.PCAP.CD,Info-Country_Name,Legal_Systems-Civil_Law,Legal_Systems-Common_Law,Legal_Systems-Customary,...,Languages-Official_language,Geography-x_coord,Geography-y_coord,Geography-Region,Geography-Sub_Region,Geography-Intermediate_Region,Geography-Region_Code,Geography-Sub_Region_Code,Geography-Intermediate_Region_Code,Economy-Income_Group
COUNTRY_PERIOD_INDEX,,,,,,,,,,,,,,,,,,,,,
AFG-1999,1999,AFG,7,NaN,NaN,NaN,AFGHANISTAN,0.0,0.0,0.0,...,Pashto/Dari,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2000,2000,AFG,7,NaN,NaN,180.188369,AFGHANISTAN,0.0,0.0,0.0,...,Pashto/Dari,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2001,2001,AFG,-,NaN,NaN,142.903364,AFGHANISTAN,0.0,0.0,0.0,...,Pashto/Dari,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2002,2002,AFG,-,NaN,NaN,182.174038,AFGHANISTAN,0.0,0.0,0.0,...,Pashto/Dari,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
AFG-2003,2003,AFG,-,9.825829,11.927498,199.643227,AFGHANISTAN,0.0,0.0,0.0,...,Pashto/Dari,33.939110,67.709953,Asia,Southern Asia,Southern Asia,142.0,34.0,0.0,Low income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE-2022,2022,ZWE,7,16.598777,21.476258,1676.821489,ZIMBABWE,1.0,1.0,1.0,...,English,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income
ZWE-2023,2023,ZWE,7,14.648601,19.968292,1592.416574,ZIMBABWE,1.0,1.0,1.0,...,English,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income
ZWE-2024,2024,ZWE,-,16.581003,18.640867,NaN,ZIMBABWE,1.0,1.0,1.0,...,English,-19.015438,29.154857,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,Lower middle income
